In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import os

os.environ['KAGGLE_USERNAME'] = "nirmalchoudhary09"
os.environ['KAGGLE_KEY'] = "08fb96674fa6f063fe431fb081d972ca"

In [ ]:
#paste api command
!kaggle datasets download -d sadmansakibmahi/plant-disease-expert

100% 6.52G/6.54G [01:37<00:00, 124MB/s]
100% 6.54G/6.54G [01:37<00:00, 71.9MB/s]


In [ ]:
!unzip '/content/drive/MyDrive/plant-disease-expert.zip' -d '/content/drive/MyDrive/plant-disease'

Streaming output truncated to the last 5000 lines.
  inflating: /content/drive/MyDrive/plant-disease/Image Data base/Image Data base/Tomato healthy/Tomato_healthy3715.jpg  
  inflating: /content/drive/MyDrive/plant-disease/Image Data base/Image Data base/Tomato healthy/Tomato_healthy3716.jpg  
  inflating: /content/drive/MyDrive/plant-disease/Image Data base/Image Data base/Tomato healthy/Tomato_healthy3717.jpg  
  inflating: /content/drive/MyDrive/plant-disease/Image Data base/Image Data base/Tomato healthy/Tomato_healthy3718.jpg  
  inflating: /content/drive/MyDrive/plant-disease/Image Data base/Image Data base/Tomato healthy/Tomato_healthy3719.jpg  
  inflating: /content/drive/MyDrive/plant-disease/Image Data base/Image Data base/Tomato healthy/Tomato_healthy372.jpg  
  inflating: /content/drive/MyDrive/plant-disease/Image Data base/Image Data base/Tomato healthy/Tomato_healthy3720.jpg  
  inflating: /content/drive/MyDrive/plant-disease/Image Data base/Image Data base/Tomato healthy

In [ ]:
import numpy as np
import pandas as pd
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'
import time
import matplotlib.pyplot as plt
import cv2
import seaborn as sns
sns.set_style('darkgrid')
import shutil
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import Dense, Activation,Dropout,Conv2D, MaxPooling2D,BatchNormalization
from tensorflow.keras.optimizers import Adam, Adamax
from tensorflow.keras.metrics import categorical_crossentropy
from tensorflow.keras import regularizers
from tensorflow.keras.models import Model

In [ ]:
sdir=r'/content/drive/MyDrive/plant-disease/Image Data base/Image Data base'

min_samples=110
filepaths = []
labels=[]
classlist=os.listdir(sdir)

for klass in classlist:
    classpath=os.path.join(sdir, klass)
    flist=os.listdir(classpath)
    if len(flist) >= min_samples:
        for f in flist:
            fpath=os.path.join(classpath,f)
            filepaths.append(fpath)
            labels.append(klass)
    else:
        print('class ', klass, ' has only', len(flist), ' samples and will not be included in dataframe')

Fseries=pd.Series(filepaths, name='filepaths')
Lseries=pd.Series(labels, name='labels')

df=pd.concat([Fseries, Lseries], axis=1)

train_df, dummy_df=train_test_split(df, train_size=.9, shuffle=True, random_state=123, stratify=df['labels'])
valid_df, test_df=train_test_split(dummy_df, train_size=.5, shuffle=True, random_state=123, stratify=dummy_df['labels'])
print('train_df length: ', len(train_df), '  test_df length: ', len(test_df), '  valid_df length: ', len(valid_df))

classes=sorted(list(train_df['labels'].unique()))
class_count = len(classes)
print('The number of classes in the dataset is: ', class_count)

groups=train_df.groupby('labels')
print('{0:^30s} {1:^13s}'.format('CLASS', 'IMAGE COUNT'))
countlist=[]
classlist=[]

for label in sorted(list(train_df['labels'].unique())):
    group=groups.get_group(label)
    countlist.append(len(group))
    classlist.append(label)
    print('{0:^30s} {1:^13s}'.format(label, str(len(group))))

# max_value=np.max(countlist)
# max_index=countlist.index(max_value)
# max_class=classlist[max_index]
# min_value=np.min(countlist)
# min_index=countlist.index(min_value)
# min_class=classlist[min_index]
# print(max_class, ' has the most images= ',max_value, ' ', min_class, ' has the least images= ', min_value)

# ht=0
# wt=0
# # select 100 random samples of train_df
# train_df_sample=train_df.sample(n=100, random_state=123,axis=0)
# for i in range (len(train_df_sample)):
#     fpath=train_df_sample['filepaths'].iloc[i]
#     img=plt.imread(fpath)
#     shape=img.shape
#     ht += shape[0]
#     wt += shape[1]
# print('average height= ', ht//100, ' average width= ', wt//100, 'aspect ratio= ', ht/wt)

class  Nitrogen deficiency in plant  has only 33  samples and will not be included in dataframe
class  Sogatella rice  has only 78  samples and will not be included in dataframe
class  Waterlogging in plant  has only 21  samples and will not be included in dataframe
class  onion  has only 60  samples and will not be included in dataframe
class  potassium deficiency in plant  has only 54  samples and will not be included in dataframe
class  tomato canker  has only 57  samples and will not be included in dataframe
train_df length:  179425   test_df length:  9969   valid_df length:  9968
The number of classes in the dataset is:  52
            CLASS               IMAGE COUNT 
       Apple Apple scab            5443     
       Apple Black rot             5368     
    Apple Cedar apple rust         2376     
        Apple healthy              3553     
Bacterial leaf blight in rice leaf      108     
     Blight in corn Leaf           3094     
      Blueberry healthy            3245     

In [ ]:
def trim(df, max_samples, min_samples, column):
    df=df.copy()
    groups=df.groupby(column)
    trimmed_df = pd.DataFrame(columns = df.columns)
    groups=df.groupby(column)
    for label in df[column].unique():
        group=groups.get_group(label)
        count=len(group)
        if count > max_samples:
            sampled_group=group.sample(n=max_samples, random_state=123,axis=0)
            trimmed_df=pd.concat([trimmed_df, sampled_group], axis=0)
        else:
            if count>=min_samples:
                sampled_group=group
                trimmed_df=pd.concat([trimmed_df, sampled_group], axis=0)
    print('after trimming, the maximum samples in any class is now ',max_samples, ' and the minimum samples in any class is ', min_samples)
    return trimmed_df

max_samples=500
min_samples=100
column='labels'
train_df= trim(train_df, max_samples, min_samples, column)

after trimming, the maximum samples in any class is now  500  and the minimum samples in any class is  100


In [ ]:
def balance(df, n, working_dir, img_size):
    df=df.copy()
    print('Initial length of dataframe is ', len(df))
    aug_dir=os.path.join(working_dir, 'aug')
    if os.path.isdir(aug_dir):
        shutil.rmtree(aug_dir)
    os.mkdir(aug_dir)
    for label in df['labels'].unique():
        dir_path=os.path.join(aug_dir,label)
        os.mkdir(dir_path)

    total=0
    gen=ImageDataGenerator(horizontal_flip=True,  rotation_range=20, width_shift_range=.2,
                                  height_shift_range=.2, zoom_range=.2)
    groups=df.groupby('labels')
    for label in df['labels'].unique():
        group=groups.get_group(label)
        sample_count=len(group)
        if sample_count< n:
            aug_img_count=0
            delta=n - sample_count
            target_dir=os.path.join(aug_dir, label)
            msg='{0:40s} for class {1:^30s} creating {2:^5s} augmented images'.format(' ', label, str(delta))
            print(msg, '\r', end='')
            aug_gen=gen.flow_from_dataframe( group,  x_col='filepaths', y_col=None, target_size=img_size,
                                            class_mode=None, batch_size=1, shuffle=False,
                                            save_to_dir=target_dir, save_prefix='aug-', color_mode='rgb',
                                            save_format='jpg')
            while aug_img_count<delta:
                images=next(aug_gen)
                aug_img_count += len(images)
            total +=aug_img_count
    print('Total Augmented images created= ', total)

    aug_fpaths=[]
    aug_labels=[]
    classlist=os.listdir(aug_dir)
    for klass in classlist:
        classpath=os.path.join(aug_dir, klass)
        flist=os.listdir(classpath)
        for f in flist:
            fpath=os.path.join(classpath,f)
            aug_fpaths.append(fpath)
            aug_labels.append(klass)
    Fseries=pd.Series(aug_fpaths, name='filepaths')
    Lseries=pd.Series(aug_labels, name='labels')
    aug_df=pd.concat([Fseries, Lseries], axis=1)
    df=pd.concat([df,aug_df], axis=0).reset_index(drop=True)
    print('Length of augmented dataframe is now ', len(df))
    return df

n=500
working_dir=r'/content/drive/MyDrive/plant-disease/'
img_size=(256,256)
train_df=balance(train_df, n, working_dir, img_size)

Initial length of dataframe is  20740
Found 270 validated image filenames.
Found 108 validated image filenames.
Found 170 validated image filenames.
Found 211 validated image filenames.
Found 132 validated image filenames.
Found 270 validated image filenames.
Found 305 validated image filenames.
Found 281 validated image filenames.
Found 305 validated image filenames.
Found 122 validated image filenames.
Found 386 validated image filenames.
Found 200 validated image filenames.
Found 329 validated image filenames.
Found 108 validated image filenames.
Found 162 validated image filenames.
Found 108 validated image filenames.
Found 165 validated image filenames.
Found 108 validated image filenames.
Total Augmented images created=  5260
Length of augmented dataframe is now  26000


In [ ]:
batch_size=20
trgen=ImageDataGenerator(horizontal_flip=True,rotation_range=20, width_shift_range=.2,
                                  height_shift_range=.2, zoom_range=.2 )
t_and_v_gen=ImageDataGenerator()
msg='{0:70s} for train generator'.format(' ')
print(msg, '\r', end='')
train_gen=trgen.flow_from_dataframe(train_df, x_col='filepaths', y_col='labels', target_size=img_size,
                                   class_mode='categorical', color_mode='rgb', shuffle=True, batch_size=batch_size)
msg='{0:70s} for valid generator'.format(' ')
print(msg, '\r', end='')
valid_gen=t_and_v_gen.flow_from_dataframe(valid_df, x_col='filepaths', y_col='labels', target_size=img_size,
                                   class_mode='categorical', color_mode='rgb', shuffle=False, batch_size=batch_size)

length=len(test_df)
test_batch_size=sorted([int(length/n) for n in range(1,length+1) if length % n ==0 and length/n<=80],reverse=True)[0]
test_steps=int(length/test_batch_size)
msg='{0:70s} for test generator'.format(' ')
print(msg, '\r', end='')
test_gen=t_and_v_gen.flow_from_dataframe(test_df, x_col='filepaths', y_col='labels', target_size=img_size,
                                   class_mode='categorical', color_mode='rgb', shuffle=False, batch_size=test_batch_size)

classes=list(train_gen.class_indices.keys())
class_indices=list(train_gen.class_indices.values())
class_count=len(classes)
labels=test_gen.labels
print ( 'test batch size: ' ,test_batch_size, '  test steps: ', test_steps, ' number of classes : ', class_count)

Found 26000 validated image filenames belonging to 52 classes.
Found 9968 validated image filenames belonging to 52 classes.
Found 9969 validated image filenames belonging to 52 classes.
test batch size:  3   test steps:  3323  number of classes :  52


In [ ]:
img_shape=(img_size[0], img_size[1], 3)
model_name='EfficientNetB3'
base_model=tf.keras.applications.efficientnet.EfficientNetB3(include_top=False, weights="imagenet",input_shape=img_shape, pooling='max')

base_model.trainable=True
x=base_model.output
x=BatchNormalization(axis=-1, momentum=0.99, epsilon=0.001 )(x)
x = Dense(256, kernel_regularizer = regularizers.l2(l = 0.016),activity_regularizer=regularizers.l1(0.006),
                bias_regularizer=regularizers.l1(0.006) ,activation='relu')(x)
x=Dropout(rate=.4, seed=123)(x)
output=Dense(class_count, activation='softmax')(x)
model=Model(inputs=base_model.input, outputs=output)
lr=.001
model.compile(Adamax(learning_rate=lr), loss='categorical_crossentropy', metrics=['accuracy'])

43941136/43941136 [==============================] - 0s 0us/step


In [ ]:
class LR_ASK(keras.callbacks.Callback):
    def __init__ (self, model, epochs,  ask_epoch):
        super(LR_ASK, self).__init__()
        self.model=model
        self.ask_epoch=ask_epoch
        self.epochs=epochs
        self.ask=True
        self.lowest_vloss=np.inf
        self.best_weights=self.model.get_weights()
        self.best_epoch=1


    def on_train_begin(self, logs=None):
        if self.ask_epoch == 0:
            print('you set ask_epoch = 0, ask_epoch will be set to 1', flush=True)
            self.ask_epoch=1
        if self.ask_epoch >= self.epochs:
            print('ask_epoch >= epochs, will train for ', epochs, ' epochs', flush=True)
            self.ask=False
        if self.epochs == 1:
            self.ask=False
        else:
            print('Training will proceed until epoch', ask_epoch,' then you will be asked to')
            print(' enter H to halt training or enter an integer for how many more epochs to run then be asked again')
        self.start_time= time.time()

    def on_train_end(self, logs=None):
        print('loading model with weights from epoch ', self.best_epoch)
        self.model.set_weights(self.best_weights)
        tr_duration=time.time() - self.start_time
        hours = tr_duration // 3600
        minutes = (tr_duration - (hours * 3600)) // 60
        seconds = tr_duration - ((hours * 3600) + (minutes * 60))
        msg = f'training elapsed time was {str(hours)} hours, {minutes:4.1f} minutes, {seconds:4.2f} seconds)'
        print (msg, flush=True)

    def on_epoch_end(self, epoch, logs=None):
        v_loss=logs.get('val_loss')
        if v_loss< self.lowest_vloss:
            self.lowest_vloss=v_loss
            self.best_weights=self.model.get_weights()
            self.best_epoch=epoch + 1
            print (f'\n validation loss of {v_loss:7.4f} is below lowest loss, saving weights from epoch {str(epoch + 1):3s} as best weights')
        else:
            print (f'\n validation loss of {v_loss:7.4f} is above lowest loss of {self.lowest_vloss:7.4f} keeping weights from epoch {str(self.best_epoch)} as best weights')

        if self.ask:
            if epoch + 1 ==self.ask_epoch:
                print('\n Enter H to end training or  an integer for the number of additional epochs to run then ask again')
                ans=input()

                if ans == 'H' or ans =='h' or ans == '0':
                    print ('you entered ', ans, ' Training halted on epoch ', epoch+1, ' due to user input\n', flush=True)
                    self.model.stop_training = True
                else:
                    self.ask_epoch += int(ans)
                    if self.ask_epoch > self.epochs:
                        print('\nYou specified maximum epochs of as ', self.epochs, ' cannot train for ', self.ask_epoch, flush =True)
                    else:
                        print ('you entered ', ans, ' Training will continue to epoch ', self.ask_epoch, flush=True)
                        lr=float(tf.keras.backend.get_value(self.model.optimizer.lr))
                        print(f'current LR is  {lr:7.5f}  hit enter to keep  this LR or enter a new LR')
                        ans=input(' ')
                        if ans =='':
                            print (f'keeping current LR of {lr:7.5f}')
                        else:
                            new_lr=float(ans)
                            tf.keras.backend.set_value(self.model.optimizer.lr, new_lr)
                            print(' changing LR to ', ans)

In [ ]:
epochs=40
ask_epoch=3
ask=LR_ASK(model, epochs,  ask_epoch)
#rlronp=tf.keras.callbacks.ReduceLROnPlateau(monitor="val_loss", factor=0.5, patience=2,verbose=1)
#callbacks = [rlronp, ask]
callbacks=[ask]

In [ ]:
history=model.fit(x=train_gen,  epochs=epochs, verbose=1, callbacks=callbacks,  validation_data=valid_gen,
               validation_steps=None,  shuffle=False,  initial_epoch=0)

Training will proceed until epoch 3  then you will be asked to
 enter H to halt training or enter an integer for how many more epochs to run then be asked again
Epoch 1/40
1300/1300 [==============================] - ETA: 0s - loss: 4.1098 - accuracy: 0.8289
 validation loss of  1.2164 is below lowest loss, saving weights from epoch 1   as best weights
1300/1300 [==============================] - 974s 694ms/step - loss: 4.1098 - accuracy: 0.8289 - val_loss: 1.2164 - val_accuracy: 0.9619
Epoch 2/40
1300/1300 [==============================] - ETA: 0s - loss: 0.8869 - accuracy: 0.9555
 validation loss of  0.5399 is below lowest loss, saving weights from epoch 2   as best weights
1300/1300 [==============================] - 848s 652ms/step - loss: 0.8869 - accuracy: 0.9555 - val_loss: 0.5399 - val_accuracy: 0.9826
Epoch 3/40
1300/1300 [==============================] - ETA: 0s - loss: 0.5718 - accuracy: 0.9730
 validation loss of  0.4371 is below lowest loss, saving weights from epoch 3  

In [ ]:
# Convert the model.
converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()

# Save the model.
with open('/content/drive/MyDrive/plant-disease-model.tflite', 'wb') as f:
  f.write(tflite_model)

In [ ]:
img_path = '/content/drive/MyDrive/Colab Notebooks/Dataset/scripting/plantvillage/PlantVillage/Potato___Early_blight/002a55fb-7a3d-4a3a-aca8-ce2d5ebc6925___RS_Early.B 8170.JPG'
img = cv2.imread(img_path)
img = cv2.resize(img,(256,256))
img = img.reshape(-1,256,256,3)

In [ ]:
classes

['Apple Apple scab',
 'Apple Black rot',
 'Apple Cedar apple rust',
 'Apple healthy',
 'Bacterial leaf blight in rice leaf',
 'Blight in corn Leaf',
 'Blueberry healthy',
 'Brown spot in rice leaf',
 'Cercospora leaf spot',
 'Cherry (including sour) Powdery mildew',
 'Cherry (including_sour) healthy',
 'Common Rust in corn Leaf',
 'Corn (maize) healthy',
 'Garlic',
 'Grape Black rot',
 'Grape Esca Black Measles',
 'Grape Leaf blight Isariopsis Leaf Spot',
 'Grape healthy',
 'Gray Leaf Spot in corn Leaf',
 'Leaf smut in rice leaf',
 'Orange Haunglongbing Citrus greening',
 'Peach healthy',
 'Pepper bell Bacterial spot',
 'Pepper bell healthy',
 'Potato Early blight',
 'Potato Late blight',
 'Potato healthy',
 'Raspberry healthy',
 'Soybean healthy',
 'Strawberry Leaf scorch',
 'Strawberry healthy',
 'Tomato Bacterial spot',
 'Tomato Early blight',
 'Tomato Late blight',
 'Tomato Leaf Mold',
 'Tomato Septoria leaf spot',
 'Tomato Spider mites Two spotted spider mite',
 'Tomato Target Spo

In [ ]:
prediction = model.predict(img)
prediction

1/1 [==============================] - 0s 41ms/step


array([[1.6504417e-03, 1.5560405e-02, 1.4382509e-03, 4.0631364e-03,
        1.5655885e-03, 1.9238307e-03, 3.8471899e-03, 3.1510934e-03,
        1.5458040e-02, 6.0696667e-04, 3.0193285e-03, 2.2309071e-03,
        1.2907461e-03, 2.3532116e-03, 1.1100112e-02, 2.5275089e-03,
        5.0974935e-03, 1.5702357e-03, 3.3498909e-03, 1.1812691e-03,
        2.4494431e-03, 1.9953030e-03, 5.4873428e-03, 2.5111921e-02,
        6.8044341e-01, 7.0035914e-03, 7.6678880e-03, 1.0337483e-03,
        4.7811274e-03, 3.6557384e-03, 1.6895613e-03, 4.3834839e-03,
        9.3523264e-02, 5.2485904e-03, 1.7067017e-03, 1.4376763e-02,
        4.4740983e-03, 2.8102065e-03, 1.4801822e-02, 1.7178146e-03,
        4.2549679e-03, 2.8686298e-03, 1.6898175e-03, 4.4860551e-03,
        4.1711912e-03, 2.5669222e-03, 1.5672093e-03, 2.7432279e-03,
        2.3563746e-03, 2.1509472e-03, 2.1298877e-03, 1.6673090e-03]],
      dtype=float32)

In [ ]:
index = np.argmax(prediction,axis=1)
disease = classes[index[0]]
print(classes[index[0]])

Potato Early blight


In [ ]:
import json

In [ ]:
f = open('/content/drive/MyDrive/plant-disease/plant diseases cure/cure.json')

data = json.load(f)
disease = disease.lower()

if disease.find(classes[index[0]]) == -1:
  print('No worry, we got your back\n\nHere is the cure\n\n')
  x = data[disease].split('.')
  for dis in x:
    print(dis+'\n')
else:
  print('Relax!, No Cure Needed\nThe Plant is healthy')
f.close()

No worry, we got your back

Here is the cure


Early blight of potato is caused by the fungal pathogen Alternaria solani

 The disease affects leaves, stems and tubers and can reduce yield, tuber size, storability of tubers, quality of fresh-market and processing tubers and marketability of the crop

Treatment of early blight includes prevention by planting potato varieties that are resistant to the disease; late maturing are more resistant than early maturing varieties

 Avoid overhead irrigation and allow for sufficient aeration between plants to allow the foliage to dry as quickly as possible

The first sign of potato blight is dark blotches on the leaves, starting at the leaf tips and edges

 They then spread, causing the foliage to shrivel and collapse

 Blotches also appear on the stems, which turn black and rot



